**Check GPU**

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Nov 30 00:23:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    43W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**Check CPU**

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
import keras as keras
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf
assert tf.__version__ >= "2.0"
import numpy as np
import os
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers, callbacks
import pandas as pd
import cv2
import matplotlib
from keras.datasets import mnist, fashion_mnist
from keras.utils import np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, MaxPooling2D, Conv2D
import tensorflow_datasets as tfds
from sklearn.cluster import KMeans

In [ ]:
column_names = ["Origin","Labeling State","All Labels","Correct Label","Type"]
df_raw = pd.DataFrame(columns=column_names)

In [ ]:
file_dir = '/content/drive/MyDrive/data/'
labels = pd.read_csv(file_dir + 'hemorrhage-labels.csv')
labels['sum'] = labels[['epidural','intraparenchymal','intraventricular','subarachnoid','subdural']].sum(axis=1)
labels = labels.drop(labels[labels['Image'] == 'ID_882cd57de'].index)#pic size 490 × 638
labels = labels.drop(labels[(labels['Image'] == 'ID_6431af929') &
                            (labels['Image'] == '20440abc6') &
                            (labels['Image'] == '1811d9bf2') &
                            (labels['Image'] == '75a698178') &
                            (labels['Image'] == '30010d605') &
                            (labels['Image'] == '34b04fff9') &
                            (labels['Image'] == '432bad415') &
                            (labels['Image'] == '3cb85bfca') &
                            (labels['Image'] == '4dad8b080')].index)#pic file broken


In [ ]:
labels

,Image,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,sum
0,ID_000012eaf,0,0,0,0,0,0,0
1,ID_000039fa0,0,0,0,0,0,0,0
2,ID_00005679d,0,0,0,0,0,0,0
3,ID_00008ce3c,0,0,0,0,0,0,0
4,ID_0000950d7,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
752798,ID_ffff82e46,0,0,0,0,0,0,0
752799,ID_ffff922b9,1,0,0,1,0,0,1
752800,ID_ffffb670a,1,0,0,0,1,0,1
752801,ID_ffffcbff8,0,0,0,0,0,0,0


In [ ]:
filename = (['brain_bone_window','brain_window','max_contrast_window','subdural_window'])

case = labels.drop(labels[labels['any'] == 0].index)

epidural = case.drop(case[(case['epidural'] == 0) | (case['sum'] != 1)].index)
epi = epidural.Image#1
intraparenchymal = case.drop(case[(case['intraparenchymal'] == 0) | (case['sum'] != 1)].index)
intrap = intraparenchymal.Image#2
intraventricular = case.drop(case[(case['intraventricular'] == 0) | (case['sum'] != 1)].index)
intrav = intraventricular.Image#3
subarachnoid = case.drop(case[(case['subarachnoid'] == 0) | (case['sum'] != 1)].index)
suba = subarachnoid.Image#4
subdural = case.drop(case[(case['subdural'] == 0) | (case['sum'] != 1)].index)
subd = subdural.Image#5
normal = labels.drop(labels[labels['any'] == 1].index)
nor = normal.Image#6

In [ ]:
case

,Image,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,sum
14,ID_0002081b6,1,0,1,0,0,0,1
24,ID_0002a38ad,1,0,0,0,1,1,2
33,ID_000346ce2,1,0,0,0,0,1,1
36,ID_00042829c,1,0,0,1,0,0,1
43,ID_0004a5701,1,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...
752755,ID_fffc60817,1,0,1,1,0,0,2
752769,ID_fffd00949,1,0,0,0,1,0,1
752783,ID_fffe2edb8,1,0,1,1,0,0,2
752799,ID_ffff922b9,1,0,0,1,0,0,1


In [ ]:
Test_Size = 500

epidural_random = epidural.sample(n=Test_Size,replace=False,random_state=150)
epi_random = epidural_random.Image
intraparenchymal_random = intraparenchymal.sample(n=Test_Size,replace=False,random_state=150)
intrap_random = intraparenchymal_random.Image
intraventricular_random = intraventricular.sample(n=Test_Size,replace=False,random_state=150)
intrav_random = intraventricular_random.Image
subarachnoid_random = subarachnoid.sample(n=Test_Size,replace=False,random_state=150)
suba_random = subarachnoid_random.Image
subdural_random = subdural.sample(n=Test_Size,replace=False,random_state=150)
subd_random = subdural_random.Image
normal_random = normal.head(500)
nor_random = normal_random.Image

# Import Data Set of 4 images

# 1 Epidural

In [ ]:
x_epi_1 = np.zeros([500, 262144])
y_epi = np.zeros(500)
for i, file in enumerate(epidural_random.Image):
    x_epi_1[i,:] = np.mean(matplotlib.image.imread(file_dir + 'epidural/' + filename[0] + '/' + file + '.jpg'),axis=2).reshape(-1)
x_epi_2 = np.zeros([500, 262144])
for i, file in enumerate(epidural_random.Image):
    x_epi_2[i,:] = np.mean(matplotlib.image.imread(file_dir + 'epidural/' + filename[1] + '/' + file + '.jpg'),axis=2).reshape(-1)
x_epi_3 = np.zeros([500, 262144])
for i, file in enumerate(epidural_random.Image):
    x_epi_3[i,:] = np.mean(matplotlib.image.imread(file_dir + 'epidural/' + filename[2] + '/' + file + '.jpg'),axis=2).reshape(-1)
x_epi_4 = np.zeros([500, 262144])
for i, file in enumerate(epidural_random.Image):
    x_epi_4[i,:] = np.mean(matplotlib.image.imread(file_dir + 'epidural/' + filename[3] + '/' + file + '.jpg'),axis=2).reshape(-1)